In [1]:
class MyProperty :
    def __init__(self, getter=None, setter=None, deleter= None) :
        self.fgetter = getter
        self.fsetter = setter
        self.fdeleter = deleter
    
    def __get__(self, instance, owner) :
        if self.fgetter is not None :
            return self.fgetter(instance)
        else :
            raise AttributeError(f" __get__ : {owner.__name__} {self.__class__.__name__} ")
    
    def __set__(self, instance , value) :
        if self.fsetter is not None :
            return self.fsetter(instance, value)
        else :
            raise AttributeError(f" __set__ : {instance.__class__.__name__} {self.__class__.__name__} ")
    
    def __delete__(self,instance) :
        if self.fdeleter is not None :
            return self.fdeleter(instance)
        else :
            raise AttributeError(f" __del__ : {owner.__class__.__name__} {self.__class__.__name__} ")
            


In [2]:
def delete(self) :
    del self.__class__.x

In [3]:
class MyClass :
    x = MyProperty(
        getter = (lambda instance : 10),
        setter = (lambda instance, value : None),
        deleter = (delete)
    )

In [4]:
m = MyClass()

In [5]:
m.__dict__

{}

In [6]:
m.x

10

In [7]:
m.x = 300

In [8]:
m.x

10

In [9]:
del m.x

In [10]:
m.x

AttributeError: 'MyClass' object has no attribute 'x'

## 일반적인 프로퍼티 처리 

In [11]:
class MyProperty2:
    def __init__(self, value):
        self._value = value

    @property
    def value(self):
        return self._value

    @value.setter
    def value(self, new_value):
        if isinstance(new_value, int):
            self._value = new_value
        else:
            print("MyProperty value must be an integer.")

    @value.deleter
    def value(self):
        print("Deleting the value.")
        del self._value

In [12]:
m2 = MyProperty2(100)

In [13]:
m2.value

100

In [14]:
del m2.value

Deleting the value.


In [15]:
m2.value

AttributeError: 'MyProperty2' object has no attribute '_value'

## 프로퍼터 클래스 정의 

In [16]:
class ClassPropertyDescriptor(object):

    def __init__(self, fget, fset=None):
        self.fget = fget
        self.fset = fset

    def __get__(self, obj, klass=None):
        if klass is None:
            klass = type(obj)
        return self.fget.__get__(obj, klass)()

    def __set__(self, obj, value):
        if not self.fset:
            raise AttributeError("can't set attribute")
        type_ = type(obj)
        return self.fset.__get__(obj, type_)(value)

    def setter(self, func):
        if not isinstance(func, (classmethod, staticmethod)):
            func = classmethod(func)
        self.fset = func
        return self



## 테코레이터 처리 

In [17]:
def classproperty(func):
    if not isinstance(func, (classmethod, staticmethod)):
        func = classmethod(func)

    return ClassPropertyDescriptor(func)

## 클래스 바녕 

In [18]:
class Bar(object):

    _bar = 1

    @classproperty
    def bar(cls):
        return cls._bar

    @bar.setter
    def bar(cls, value):
        cls._bar = value



In [19]:
# test instance instantiation
foo = Bar()
assert foo.bar == 1

baz = Bar()
assert baz.bar == 1

# test static variable
baz.bar = 5
assert foo.bar == 5

# test setting variable on the class
Bar.bar = 50
assert baz.bar == 50
assert foo.bar == 50